In [24]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer

In [25]:
demo_df = pd.read_csv("c:/sujan/ny taxies data/train.csv")

In [8]:
demo_df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [26]:
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  object 
 1   vendor_id           1458644 non-null  int64  
 2   pickup_datetime     1458644 non-null  object 
 3   dropoff_datetime    1458644 non-null  object 
 4   passenger_count     1458644 non-null  int64  
 5   pickup_longitude    1458644 non-null  float64
 6   pickup_latitude     1458644 non-null  float64
 7   dropoff_longitude   1458644 non-null  float64
 8   dropoff_latitude    1458644 non-null  float64
 9   store_and_fwd_flag  1458644 non-null  object 
 10  trip_duration       1458644 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


Import the dataset: Import the dataset into your programming environment using a library such as Pandas, NumPy, or TensorFlow.

Data Cleaning: Check the dataset for missing values, inconsistent data, and outliers, and handle them accordingly. This might involve filling in missing values, removing duplicates, or removing outliers.

Data Transformation: Transform the data to make it suitable for analysis. This might involve converting categorical variables to numerical values, scaling or normalizing numerical data, or converting date and time values to a standard format.

Feature Extraction: Extract meaningful features from the data that can be used in the analysis. This might involve feature selection, dimensionality reduction, or creating new features using domain knowledge.

Splitting the dataset: Split the dataset into training, validation, and test sets to evaluate the performance of the machine learning model.

Encoding: Convert categorical variables into numeric variables, One-Hot-Encoding or Label Encoding depending on the data and model.

Scaling: Scale the numerical variables to have zero mean and unit variance.

Handling Outliers: Identify and handle outliers in the dataset. This can be done using statistical techniques or domain knowledge.

Handling Imbalanced Data: If the dataset is imbalanced, balance the dataset by either undersampling, oversampling, or using techniques such as SMOTE.

Data Visualization: Visualize the data to gain insights into the data and to detect patterns.

In [19]:
def preprocess(data):
    #checking missing values
    percent_missing = data.isnull().sum() * 100 / data.shape[0]
    #dropping columns if missing percentage is more than 30
    for i in range(len(data.columns)):
        if percent_missing[i] >0.3:
            data.drop(demo_df.columns[i],axis=1,inplace=True)
    #getting numerical and categorical variables
    numerical_columns = [x for x in data.columns if train[x].dtype != 'object']
    cat_columns = [x for x in data.columns if x not in numerical_columns]
    #Imputing using KNN Imputer
    imputer = KNNImputer(n_neighbors=2)
    


In [13]:
percent_missing = demo_df.isnull().sum() * 100 / demo_df.shape[0]
percent_missing


id                    0.0
vendor_id             0.0
pickup_datetime       0.0
dropoff_datetime      0.0
passenger_count       0.0
pickup_longitude      0.0
pickup_latitude       0.0
dropoff_longitude     0.0
dropoff_latitude      0.0
store_and_fwd_flag    0.0
trip_duration         0.0
dtype: float64

In [22]:
demo_df.isnull().sum()/demo_df.shape[0]

passenger_count       0.0
pickup_longitude      0.0
pickup_latitude       0.0
dropoff_longitude     0.0
dropoff_latitude      0.0
store_and_fwd_flag    0.0
trip_duration         0.0
dtype: float64

In [23]:
demo_df.shape

(1458644, 7)

In [27]:
numerical_columns = [x for x in demo_df.columns if demo_df[x].dtype != 'object']

In [28]:
numerical_columns

['vendor_id',
 'passenger_count',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'trip_duration']

In [30]:
demo_df_num = demo_df[numerical_columns]
imputer = KNNImputer(n_neighbors=2)
imputer.fit_transform(demo_df_num)

array([[ 2.00000000e+00,  1.00000000e+00, -7.39821548e+01, ...,
        -7.39646301e+01,  4.07656021e+01,  4.55000000e+02],
       [ 1.00000000e+00,  1.00000000e+00, -7.39804153e+01, ...,
        -7.39994812e+01,  4.07311516e+01,  6.63000000e+02],
       [ 2.00000000e+00,  1.00000000e+00, -7.39790268e+01, ...,
        -7.40053329e+01,  4.07100868e+01,  2.12400000e+03],
       ...,
       [ 2.00000000e+00,  1.00000000e+00, -7.39591293e+01, ...,
        -7.40044327e+01,  4.07073708e+01,  7.64000000e+02],
       [ 1.00000000e+00,  1.00000000e+00, -7.39820786e+01, ...,
        -7.39746323e+01,  4.07571068e+01,  3.73000000e+02],
       [ 1.00000000e+00,  1.00000000e+00, -7.39795380e+01, ...,
        -7.39728088e+01,  4.07905846e+01,  1.98000000e+02]])

In [31]:
demo_df_num

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,2,1,-73.982155,40.767937,-73.964630,40.765602,455
1,1,1,-73.980415,40.738564,-73.999481,40.731152,663
2,2,1,-73.979027,40.763939,-74.005333,40.710087,2124
3,2,1,-74.010040,40.719971,-74.012268,40.706718,429
4,2,1,-73.973053,40.793209,-73.972923,40.782520,435
...,...,...,...,...,...,...,...
1458639,2,4,-73.982201,40.745522,-73.994911,40.740170,778
1458640,1,1,-74.000946,40.747379,-73.970184,40.796547,655
1458641,2,1,-73.959129,40.768799,-74.004433,40.707371,764
1458642,1,1,-73.982079,40.749062,-73.974632,40.757107,373


In [33]:
demo_df[numerical_columns]

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,2,1,-73.982155,40.767937,-73.964630,40.765602,455
1,1,1,-73.980415,40.738564,-73.999481,40.731152,663
2,2,1,-73.979027,40.763939,-74.005333,40.710087,2124
3,2,1,-74.010040,40.719971,-74.012268,40.706718,429
4,2,1,-73.973053,40.793209,-73.972923,40.782520,435
...,...,...,...,...,...,...,...
1458639,2,4,-73.982201,40.745522,-73.994911,40.740170,778
1458640,1,1,-74.000946,40.747379,-73.970184,40.796547,655
1458641,2,1,-73.959129,40.768799,-74.004433,40.707371,764
1458642,1,1,-73.982079,40.749062,-73.974632,40.757107,373
